In [1]:
import logging
from fairlib import dataloaders
import numpy as np
from pathlib import Path
import torch
import random
import sys
sys.path.append('../')

In [2]:
from fairlib.src.base_options import BaseOptions
from fairlib.src import networks

In [3]:
options = BaseOptions()

state = options.get_state(
    args={
        "data_dir": r'/data/bios',

    }, 
    conf_file=r"./Experimental_results/Leakage_estimation/Bios/Vanilla_1/opt.yaml",
    silence=True)

INFO:root:Unexpected args: ['--ip=127.0.0.1', '--stdin=9020', '--control=9018', '--hb=9017', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"4e8ccfc2-bc39-4c52-b710-824eb123111c"', '--shell=9019', '--transport="tcp"', '--iopub=9021']
INFO:root:Logging to ./results/vanilla\Bios_gender\4\output.log


2022-07-02 23:59:37 [INFO ]  ======================================== 2022-07-02 23:59:37 ========================================
2022-07-02 23:59:37 [INFO ]  Base directory is ./results/vanilla\Bios_gender\4
Loaded data shapes: (257478, 768), (257478,), (257478,)
Loaded data shapes: (39642, 768), (39642,), (39642,)
Loaded data shapes: (99069, 768), (99069,), (99069,)


In [4]:
def load_trained_model(model, checkpoint_dir, device):
    checkpoint_PATH = Path(checkpoint_dir) / 'BEST_checkpoint.pth.tar'
    model.load_state_dict(torch.load(checkpoint_PATH)["model"])
    model.to(device)
    model.eval()
    return model

## Retrive Experimental Results and Model Checopints

In [5]:
from fairlib.src.analysis.utils import retrive_exp_results

In [6]:
methods = ["FSCL_0", "FSCL_EO", "GatedBT", "GDMean", "R_DAdv", "Original_FairBatch", "Vanilla"]

In [7]:
import os
# find folders for each run
def get_dir(results_dir, project_dir, checkpoint_dir, checkpoint_name, model_id):
    """retrive logs for experiments

    Args:
        results_dir (str): dir to the saved experimental results
        project_dir (str): experiment type identifier, e.g., final, hypertune, dev. Same as the arguments.
        checkpoint_dir (str): dir to checkpoints, `models` by default.
        checkpoint_name (str):  checkpoint_epoch{num_epoch}.ptr.gz
        model_id (str): read all experiment start with the same model_id. E.g., "Adv" when tuning hyperparameters for standard adversarial

    Returns:
        list: a list of dictionaries, where each dict contains the information for a experiment.
    """
    results_dir = Path(results_dir)
    project_dir = Path(project_dir)
    checkpoint_dir = Path(checkpoint_dir)

    exps = []
    for root, dirs, files in os.walk(results_dir / project_dir):
        # for file in files:
            # if file.endswith(".txt"):
                # print(os.path.join(root, file))
        for dir in dirs:
            if dir.startswith(model_id):
                _dirs = Path(os.path.join(root, dir))
                
                _checkpoints_dirs = []
                for root2, dirs2, files2 in os.walk(_dirs / checkpoint_dir):
                    for file2 in files2:
                        if file2.startswith(checkpoint_name):
                            _dirs2 = os.path.join(root2, file2)
                            _checkpoints_dirs.append(_dirs2)
                exps.append(
                    {
                        "opt_dir":os.path.join(root, dir),
                        "opt":{},
                        "dir":_checkpoints_dirs,
                    }
                )
    return exps

In [8]:
retrived_runs = {}

for method in methods:
    retrived_runs[method] = get_dir(
        results_dir=r"G:\Experimental_results\Leakage_estimation",
        project_dir=r"Bios",
        checkpoint_dir="models",
        checkpoint_name="checkpoint_epoch",
        model_id=method
    )

In [16]:
for method in methods:

    # Iterate over the each random run
    for _run in retrived_runs[method]:
        selected_model = retrive_exp_results(
            exp = _run,
            GAP_metric_name="TPR_GAP",
            Performance_metric_name="accuracy",
            selection_criterion = "DTO",
            index_column_names = []
        )

        infos = selected_model["opt_dir"].split("\\")
        # print(infos)
        # continue

        checkopoints = torch.load("\\".join(infos)+"\\models\\checkpoint_epoch{}.00.pth.tar".format(selected_model["epoch"]))

        options = BaseOptions()

        _state = options.get_state(
            args={
                "data_dir": r'/data/bios',

            }, 
            conf_file=selected_model["opt_dir"]+r"\opt.yaml",
            silence=True)

        # Init the main task model and load the trained parameters
        model = networks.get_main_model(_state)
        model.load_state_dict(checkopoints["model"])
        model.to(_state.device)
        model.eval()

        # Extract hidden representations
        train_hidden, train_labels, train_private_labels, train_regression_labels = model.extract_hidden_representations("train")
        dev_hidden, dev_labels, dev_private_labels, dev_regression_labels = model.extract_hidden_representations("dev")
        test_hidden, test_labels, test_private_labels, test_regression_labels = model.extract_hidden_representations("test")

        # Prepare dataloader
        class CustomizedDataset(dataloaders.utils.BaseDataset):

            def load_data(self):

                if self.split == "train":
                    self.X = train_hidden.astype(np.float32)
                    self.y = list(train_private_labels)
                    self.protected_label = list(train_private_labels)
                elif self.split == "dev":
                    self.X = dev_hidden.astype(np.float32)
                    self.y = list(dev_private_labels)
                    self.protected_label = list(dev_private_labels)
                elif self.split == "test":
                    self.X = test_hidden.astype(np.float32)
                    self.y = list(test_private_labels)
                    self.protected_label = list(test_private_labels)
                else:
                    raise
        
        # Data from discriminator training
        customized_train_data = CustomizedDataset(args=state, split="train")
        customized_dev_data = CustomizedDataset(args=state, split="dev")
        customized_test_data = CustomizedDataset(args=state, split="test")

        # DataLoader Parameters
        tran_dataloader_params = {
                'batch_size': state.batch_size,
                'shuffle': True,
                'num_workers': state.num_workers}

        eval_dataloader_params = {
                'batch_size': state.test_batch_size,
                'shuffle': False,
                'num_workers': state.num_workers}

        # init dataloader for attacker
        customized_training_generator = torch.utils.data.DataLoader(customized_train_data, **tran_dataloader_params)
        customized_validation_generator = torch.utils.data.DataLoader(customized_dev_data, **eval_dataloader_params)
        customized_test_generator = torch.utils.data.DataLoader(customized_test_data, **eval_dataloader_params)

        # Train the dicriminator
        discriminator_args = {

            "device_id":0,
            # Give a name to the exp, which will be used in the path
            "exp_id":infos[3]+"_leakage_"+infos[4],

            "emb_size": 300,
            "lr": 0.001,
            "batch_size": 128,
            "hidden_size": 300,
            "n_hidden": 1,
            "activation_function": "ReLu",
            # "activation_function": "Tanh",
            "base_seed": random.randint(0, 1e+5),

            "num_classes": 2,
            "num_groups": 2, # Balck; White; and Other
        }

        # Init the argument
        discriminator_options = BaseOptions()
        discriminator_state = discriminator_options.get_state(args=discriminator_args, silence=True)

        discriminator = networks.classifier.MLP(discriminator_state)
        discriminator

        discriminator.train_self(
            train_generator = customized_training_generator,
            dev_generator = customized_validation_generator,
            test_generator = customized_test_generator,
        )

2022-06-29 12:18:44 [INFO ]  Unexpected args: ['--ip=127.0.0.1', '--stdin=9015', '--control=9011', '--hb=9010', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"9ace1b00-8d28-4db0-a1f3-5ae00a5c01d2"', '--shell=9014', '--transport="tcp"', '--iopub=9016']
2022-06-29 12:18:44 [INFO ]  Logging to /data/cephfs/punim0478/xudongh1/experimental_results/Fair_NLP_Classification\FairSCL\Bios_gender\FSCL_0.00011885022274370189_0\output.log
2022-06-29 12:18:44 [INFO ]  ======================================== 2022-06-29 12:18:44 ========================================
2022-06-29 12:18:44 [INFO ]  Base directory is /data/cephfs/punim0478/xudongh1/experimental_results/Fair_NLP_Classification\FairSCL\Bios_gender\FSCL_0.00011885022274370189_0
Loaded data shapes: (257478, 768), (257478,), (257478,)
Loaded data shapes: (39642, 768), (39642,), (39642,)
Loaded data shapes: (99069, 768), (99069,), (99069,)
2022-06-29 12:18:49 [INFO ]  MLP( 
2022-06-29 12:18:49 [INFO ]    (output_layer): Linear(

In [9]:
retrived_runs_INLP = get_dir(
    results_dir=r"/Experimental_results/Leakage_estimation",
    project_dir=r"Bios",
    checkpoint_dir="models",
    checkpoint_name="INLP_checkpoint_epoch",
    model_id="INLP"
)

In [10]:
from fairlib.src.analysis.utils import retrive_all_exp_results

In [11]:
INLP_selected_model = retrive_all_exp_results(
    exp = retrived_runs_INLP[0],
    GAP_metric_name="TPR_GAP",
    Performance_metric_name="accuracy",
    index_column_names = []
)

In [12]:
INLP_selected_model = INLP_selected_model.sort_values("epoch")
INLP_selected_model

,epoch,dev_DTO,test_DTO,dev_performance,dev_fairness,test_performance,test_fairness,opt_dir
0,0,0.136492,0.134500,0.818551,0.847438,0.820640,0.849193,G:\Experimental_results\Leakage_estimation\Bio...
1,1,0.126546,0.128017,0.819535,0.857176,0.821862,0.855524,G:\Experimental_results\Leakage_estimation\Bio...
112,2,0.130026,0.140576,0.812598,0.854171,0.815472,0.843527,G:\Experimental_results\Leakage_estimation\Bio...
222,3,0.136841,0.134237,0.819106,0.847081,0.822487,0.849410,G:\Experimental_results\Leakage_estimation\Bio...
233,4,0.125488,0.137334,0.821351,0.858195,0.822851,0.846368,G:\Experimental_results\Leakage_estimation\Bio...
...,...,...,...,...,...,...,...,...
217,294,0.421095,0.424053,0.476086,0.956982,0.474689,0.963207,G:\Experimental_results\Leakage_estimation\Bio...
218,295,0.508545,0.512857,0.403789,0.968796,0.401357,0.968536,G:\Experimental_results\Leakage_estimation\Bio...
219,296,0.563778,0.567615,0.358307,0.976869,0.356126,0.979676,G:\Experimental_results\Leakage_estimation\Bio...
220,297,0.608046,0.609426,0.321982,0.972921,0.321705,0.976166,G:\Experimental_results\Leakage_estimation\Bio...


In [21]:
import seaborn as sns

In [25]:
selected_iteration = list(INLP_selected_model[INLP_selected_model["dev_DTO"] == min(INLP_selected_model["dev_DTO"])]["epoch"])[0]
selected_iteration

172

In [26]:
projection_matrices = torch.load(r"G:\Experimental_results\Leakage_estimation\Bios\INLP_True_None_0_0\models\projection_matrices.pth.tar")

In [29]:
options = BaseOptions()

_state = options.get_state(
    args={
            "data_dir": r'D:\Project\Minding_Imbalance_in_Discriminator_Training\data\bios',

    }, 
    conf_file=INLP_selected_model["opt_dir"][0]+r"\opt.yaml",
    silence=True)

# Init the main task model and load the trained parameters
model = networks.get_main_model(_state)
model.load_state_dict(checkopoints["model"])
model.to(_state.device)
model.eval()


2022-06-29 16:59:31 [INFO ]  Unexpected args: ['--ip=127.0.0.1', '--stdin=9015', '--control=9011', '--hb=9010', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"9ace1b00-8d28-4db0-a1f3-5ae00a5c01d2"', '--shell=9014', '--transport="tcp"', '--iopub=9016']
2022-06-29 16:59:31 [INFO ]  Logging to ./results/hypertune7\Bios_gender\INLP_True_None_0_0\output.log
2022-06-29 16:59:31 [WARNING]  Log file already exists, will append
2022-06-29 16:59:31 [INFO ]  ======================================== 2022-06-29 16:59:31 ========================================
2022-06-29 16:59:31 [INFO ]  Base directory is ./results/hypertune7\Bios_gender\INLP_True_None_0_0
2022-06-29 16:59:31 [WARNING]  ./results/hypertune7\Bios_gender\INLP_True_None_0_0\opt.yaml already exists, moved to ./results/hypertune7\Bios_gender\INLP_True_None_0_0\old_opts\opt_2022_06_29__16_58_35.yaml
Loaded data shapes: (257478, 768), (257478,), (257478,)
Loaded data shapes: (39642, 768), (39642,), (39642,)
Loaded data shap

MLP(
  (output_layer): Linear(in_features=300, out_features=28, bias=True)
  (AF): Tanh()
  (hidden_layers): ModuleList(
    (0): Linear(in_features=768, out_features=300, bias=True)
    (1): Tanh()
    (2): Linear(in_features=300, out_features=300, bias=True)
    (3): Tanh()
  )
  (criterion): CrossEntropyLoss()
)

In [30]:
# Extract hidden representations
train_hidden, train_labels, train_private_labels, train_regression_labels = model.extract_hidden_representations("train")
dev_hidden, dev_labels, dev_private_labels, dev_regression_labels = model.extract_hidden_representations("dev")
test_hidden, test_labels, test_private_labels, test_regression_labels = model.extract_hidden_representations("test")

In [31]:
from fairlib.src.networks.INLP.debias import get_projection_to_intersection_of_nullspaces

In [32]:
P = get_projection_to_intersection_of_nullspaces(projection_matrices[1][:selected_iteration], input_dim=train_hidden.shape[1])


train_hidden = P.dot(train_hidden.T).T
dev_hidden = P.dot(dev_hidden.T).T
test_hidden = P.dot(test_hidden.T).T

In [33]:
# Prepare dataloader
class CustomizedDataset(dataloaders.utils.BaseDataset):

    def load_data(self):
        if self.split == "train":
            self.X = train_hidden.astype(np.float32)
            self.y = list(train_private_labels)
            self.protected_label = list(train_private_labels)
        elif self.split == "dev":
            self.X = dev_hidden.astype(np.float32)
            self.y = list(dev_private_labels)
            self.protected_label = list(dev_private_labels)
        elif self.split == "test":
            self.X = test_hidden.astype(np.float32)
            self.y = list(test_private_labels)
            self.protected_label = list(test_private_labels)
        else:
            raise

# Data from discriminator training
customized_train_data = CustomizedDataset(args=state, split="train")
customized_dev_data = CustomizedDataset(args=state, split="dev")
customized_test_data = CustomizedDataset(args=state, split="test")
# DataLoader Parameters
tran_dataloader_params = {
        'batch_size': state.batch_size,
        'shuffle': True,
        'num_workers': state.num_workers}
eval_dataloader_params = {
        'batch_size': state.test_batch_size,
        'shuffle': False,
        'num_workers': state.num_workers}
# init dataloader for attacker
customized_training_generator = torch.utils.data.DataLoader(customized_train_data, **tran_dataloader_params)
customized_validation_generator = torch.utils.data.DataLoader(customized_dev_data, **eval_dataloader_params)
customized_test_generator = torch.utils.data.DataLoader(customized_test_data, **eval_dataloader_params)

Loaded data shapes: (257478, 300), (257478,), (257478,)
Loaded data shapes: (39642, 300), (39642,), (39642,)
Loaded data shapes: (99069, 300), (99069,), (99069,)


In [34]:
for i in range(5):
    # Train the dicriminator
    discriminator_args = {
        "device_id":0,
        # Give a name to the exp, which will be used in the path
        "exp_id":"Bios_leakage_INLP_{}".format(i),
        "emb_size": 300,
        "lr": 0.001,
        "batch_size": 128,
        "hidden_size": 300,
        "n_hidden": 1,
        "activation_function": "ReLu",
        # "activation_function": "Tanh",
        "base_seed": random.randint(0, 1e+5),
        "num_classes": 2,
        "num_groups": 2, # Balck; White; and Other
    }
    # Init the argument
    discriminator_options = BaseOptions()
    discriminator_state = discriminator_options.get_state(args=discriminator_args, silence=True)
    discriminator = networks.classifier.MLP(discriminator_state)
    discriminator
    discriminator.train_self(
        train_generator = customized_training_generator,
        dev_generator = customized_validation_generator,
        test_generator = customized_test_generator,
    )

2022-06-29 17:00:22 [INFO ]  Unexpected args: ['--ip=127.0.0.1', '--stdin=9015', '--control=9011', '--hb=9010', '--Session.signature_scheme="hmac-sha256"', '--Session.key=b"9ace1b00-8d28-4db0-a1f3-5ae00a5c01d2"', '--shell=9014', '--transport="tcp"', '--iopub=9016']
2022-06-29 17:00:22 [INFO ]  Logging to ./results/dev\Moji\Bios_leakage_INLP_0\output.log
2022-06-29 17:00:22 [INFO ]  ======================================== 2022-06-29 17:00:22 ========================================
2022-06-29 17:00:22 [INFO ]  Base directory is ./results/dev\Moji\Bios_leakage_INLP_0
2022-06-29 17:00:22 [INFO ]  Exception type : FileNotFoundError 
2022-06-29 17:00:22 [INFO ]  Exception message : [Errno 2] No such file or directory: 'data\\deepmoji\\train/pos_pos.npy'
2022-06-29 17:00:22 [INFO ]  Stack trace : ['File : d:\\project\\fair_nlp_classification\\fairlib\\src\\base_options.py , Line : 486, Func.Name : set_state, Message : train_iterator, dev_iterator, test_iterator = dataloaders.get_dataloaders

In [35]:
import statistics

In [36]:
for method in (methods+["INLP"]):
    scores = []
    for root, dirs, files in os.walk(r"D:\Project\Dev_Fairness_NLP\dev\results\dev\Moji"):
        for dir in dirs:
            if dir.startswith("Bios_leakage_{}".format(method)):
                _dirs = Path(os.path.join(root, dir))
                # print(_dirs)
                _best_results = torch.load(Path(_dirs) / "models" / "BEST_checkpoint.pth.tar")["test_evaluations"]["accuracy"]
                scores.append(_best_results)
    print(method, round(statistics.mean(scores)*100, 2), round(statistics.stdev(scores)*100, 2))

FSCL_0 76.31 1.5
FSCL_EO 84.89 3.41
GatedBT 100.0 0.0
GDMean 97.22 0.48
R_DAdv 88.61 4.55
Original_FairBatch 98.02 0.3
Vanilla 98.1 0.35
INLP 97.63 0.05
